<a href="https://colab.research.google.com/github/jsebdev/Stock_Predictor/blob/main/stock_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/projects/Stock_Predicter/v1'
%cd $project_path

Mounted at /content/drive
/content/drive/MyDrive/projects/Stock_Predicter/v1


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
import yfinance as yfin
import tensorflow as tf
import os
import re

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM


# Get Data

In [3]:
# Select a company for now
ticker = 'AAPL'

start = dt.datetime(2013,1,1)
end = dt.datetime(2023,4,5)

In [4]:
yfin.pdr_override()
data = web.data.get_data_yahoo(ticker, start, end)


[*********************100%***********************]  1 of 1 completed


# Preprocess_data

In [5]:
def create_remove_columns(data):
  # create jump column
  data = pd.DataFrame.copy(data)
  data['Jump'] = data['Open'] - data['Close'].shift(1)
  data['Jump'].fillna(0, inplace=True)
  # data = data.reindex(columns=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Jump'])
  data.insert(0,'Jump', data.pop('Jump'))
  return data

In [6]:
def normalize_data(data, scaler=None):
  the_data = pd.DataFrame.copy(data)
  # substract the open value to all columns but the first one and the last one which are "Jump" and "Volume"
  the_data.iloc[:, 1:-1] = the_data.iloc[:,1:-1] - the_data['Open'].values[:, np.newaxis]
  # print('the_data')
  # print(the_data)

  the_data.pop('Open')
  # todo save an csv with the values for the scaler
  if scaler is None:
    # Create the scaler
    values = np.abs(the_data.values)
    max_value = np.max(values[:,:-1])
    max_volume = np.max(values[:,-1])
    def scaler(d):
      data = pd.DataFrame.copy(d)
      print('max_value: ', max_value)
      print('max_volume: ', max_volume)
      data.iloc[:, :-1] = data.iloc[:,:-1].apply(lambda x: x/max_value)
      data.iloc[:, -1] = data.iloc[:,-1].apply(lambda x: x/max_volume)
      return data
    def decoder(values):
      decoded_values = values * max_value
      return decoded_values
  else:
    decoder = None
  
  normalized_data = scaler(the_data)

  return normalized_data, scaler, decoder




In [7]:
def create_training_data(norm_data):
  prediction_days = 500
  
  x_train_list = []
  y_train_list = []
  print('shape norm_data')
  print(norm_data.shape)
  
  for i in range(prediction_days, len(norm_data)):
    x_train_list.append(norm_data.iloc[i-prediction_days:i])
    y_train_list.append(norm_data.iloc[i-prediction_days+1:i+1,0:4])
  
  x_train = np.array(x_train_list)
  y_train = np.array(y_train_list)
  return x_train, y_train

In [8]:
x = np.random.randint(5, size=(5,6))
print(x)
print(x[2:4, 0:4])

[[0 3 4 2 4 1]
 [4 4 1 0 3 4]
 [0 3 3 3 2 0]
 [3 1 3 3 0 4]
 [1 0 0 1 0 3]]
[[0 3 3 3]
 [3 1 3 3]]


In [9]:
#Make all the preprocesing
def preprocessing(data, scaler=None):
  # print(data.head(3))
  data_0 = create_remove_columns(data)
  # print(data_0.head(3))
  #todo: save the_scaler somehow to use in new runtimes
  norm_data, scaler, decoder = normalize_data(data_0, scaler=scaler)
  # print(norm_data.head(3))
  x_train, y_train = create_training_data(norm_data)
  # print(x_train.shape, y_train.shape)
  return x_train, y_train, scaler, decoder

In [10]:
x_train, y_train, scaler, decoder = preprocessing(data)

max_value:  10.589996337890625
max_volume:  1460852400.0
shape norm_data
(2582, 6)


In [11]:
print(x_train.shape)
print(y_train.shape)

(2082, 500, 6)
(2082, 500, 4)


# Model

## Create Model

In [12]:
def create_model():
  model = Sequential()
  model.add(LSTM(units=1024, return_sequences=True, input_shape=(None,x_train.shape[-1],)))
  # model.add(Dropout(0.2))
  model.add(LSTM(units=1024, return_sequences=True))
  # model.add(Dropout(0.2))
  model.add(LSTM(units=1024, return_sequences=True))
  model.add(Dense(4))
  return model

model = create_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 1024)        4222976   
                                                                 
 lstm_1 (LSTM)               (None, None, 1024)        8392704   
                                                                 
 lstm_2 (LSTM)               (None, None, 1024)        8392704   
                                                                 
 dense (Dense)               (None, None, 4)           4100      
                                                                 
Total params: 21,012,484
Trainable params: 21,012,484
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
# model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

In [14]:
if False:
  model.load_weights('./training_checkpoints_20230408062729/ckpt_epoch24_loss0.00024580140598118305')

## Model Train

In [15]:
print(x_train.shape)
print(y_train.shape)

(2082, 500, 6)
(2082, 500, 4)


In [16]:
# Change to False to avoid trainging the model
to_train = True
if to_train:
# if True:
  # Directory where the checkpoints will be saved
  checkpoint_dir = './training_checkpoints_'+dt.datetime.now().strftime("%Y%m%d%H%M%S")
  # Name of the checkpoint files
  # checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_epoch{epoch}_loss{loss}")
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
  
  checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_prefix,
      save_weights_only=True,
      monitor="loss", mode="min",
      save_best_only=True)

  model.fit(x_train, y_train, epochs=25, batch_size=32, callbacks=[checkpoint_callback])


# Testing a model

In [17]:
#print trainings directories to pick one
!ls -ld training_checkpoints_*/

drwx------ 2 root root 4096 Apr  8 07:34 training_checkpoints_20230408073359/


In [18]:
model = create_model()

In [28]:
# if checkpoint_dir does not exists, select the one stated in the except block
try:
  checkpoint_dir
except NameError: 
  checkpoint_dir = 'training_checkpoints_20230408073359'

def load_weights(epoch=None):
  if epoch is None:
    weights_file = tf.train.latest_checkpoint(checkpoint_dir)
  else:
    with os.scandir(checkpoint_dir) as entries:
      for entry in entries:
        if re.search(f'^ckpt_epoch{epoch}_.*\.index', entry.name):
          weights_file = checkpoint_dir + '/'+ entry.name[:-6]

  print('weights_file')
  print(weights_file)
  model.load_weights(weights_file)
  return model

model = load_weights(epoch=None)
model_filepath = 'saved_model'
model.save(model_filepath)

weights_file
training_checkpoints_20230408073359/ckpt


In [24]:
test_start = dt.datetime(2016,1,1)
test_end = dt.datetime(2023,4,5)
ticker = 'AAPL'

yfin.pdr_override()
test_data = web.data.get_data_yahoo(ticker, test_start, test_end)

[*********************100%***********************]  1 of 1 completed


In [29]:
load_whole_model = False
if load_whole_model:
  model = tf.keras.models.load_model(model_filepath)

In [30]:
# def close_tester(model, test_data, scaler=None):
scaler = scaler
test_x_train, test_y_train, _, _ = preprocessing(test_data, scaler=scaler)
print(test_x_train.shape)
print(test_y_train.shape)
results = model.predict(test_x_train)
# the results are tensors of 4 numbers, Jump, High, Low, and Close respectively

# close_tester(test_model, test_data, scaler=the_scaler)


max_value:  10.589996337890625
max_volume:  1460852400.0
shape norm_data
(1826, 6)
(1326, 500, 6)
(1326, 500, 4)
42/42 [==============================] - 12s 270ms/step


In [31]:
right_counter = 0
wrong_counter = 0
no_action_counter = 0
# for result, expected in zip(results[:2], test_y_train[:2]):
for result, expected in zip(results[:], test_y_train[:]):
  # print(result)
  # print(expected)
  comparer = result[-1][3] * expected[-1][3]
  if comparer > 0:
    right_counter += 1
  elif comparer == 0:
    no_action_counter
  elif comparer < 0:
    wrong_counter += 1

  # print('expected: ', decoder(expected))
  # print('result: ', decoder(result))

print('right_counter :', right_counter)
print('no_action_counter :',no_action_counter)
print('wrong_counter :', wrong_counter)
print('success rate: {}%'.format(right_counter*100/len(results)))

right_counter : 1300
no_action_counter : 0
wrong_counter : 22
success rate: 98.03921568627452%


In [32]:
test_data.iloc[500,:]

Open         4.252500e+01
High         4.269500e+01
Low          4.242750e+01
Close        4.265000e+01
Adj Close    4.049405e+01
Volume       8.599280e+07
Name: 2017-12-27 00:00:00, dtype: float64